In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
clothes = pd.read_csv('Clothing-Reviews.csv')
clothes.head()

,asin,title,review_userId,review_score,review_summary,review_text
0,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A174NPQZ1EABX1,5,"Fundamental of death metal, do not miss","A roaring onslaught of streaming sound, this a..."
1,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A17ZI8VKZJFOV8,5,Great Classic,One of incantations best recordings. Im a fan ...
2,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A2F9LXG1QEJ855,4,Very Original Debut of one of the most striden...,"This is an early, and rough-sounding look at t..."
3,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A2FGXHSHF0OD17,5,No secret here,Whats to say about this album? Quintessential ...
4,B004GSI2OS,Featherlite Ladies' Long Sleeve Stain Resistan...,A2O2E8BLB7VW1U,5,Brings a smile to my face...,...Whenever i listing to an album by incantati...


In [3]:
clothes =clothes.drop(['review_summary', 'review_text',], axis=1)

In [4]:
user_matrix = clothes.pivot_table(index = 'review_userId', columns = 'title', values = 'review_score').fillna(0)

In [5]:
user_matrix

title,"""I Workout To Burn off The Crazy"", Tri Blend Tank Top, Funny Workout Fitness Shirt Sleeveless for Women (2XL, Shocking Pink)","""Life is a Journey"" Self-Help DIY T-Shirt (Women,White,Female X-Large)",#8 Dale Earnhardt Jr. Ladies Reverse Stitch Pants -XXL,1 BY O'2nd Womens LAO LIN Cut Out Detail Short Sleeve Top Sz 4 White 240367F,1 World Sarongs Womens Animal Print Tunic Cover-Up in Small,1533-G Yoga Racerback Ganesha Print Tank Top-XL-Black,1533-G Yoga Racerback Ganesha Tank-MD-White,1893 Blouse with Surplice Front Pattern,1915 Lime Green Taffeta Blouse Pattern,"2 Pack Active Basic Women's Basic Cami Tank Top Small Black, White",...,thistle purple Lao Laos Sleeveless Blouse Classic Neckline Tops Size 38 M SL38E,"underfell skele bros Women's DIY Customize T-shirt,Black (X-Large,Black,X-large,black)",vihaan impex kurtis for women indian kurti for women kurtas for women indian kurtis,virblatt batwing top as hippie clothes for women - Namasté,wallpapher Women's Medium Tank Cami Printed Sharkbit Top Black M,western turquoise blouse with front and back yokes size medium 211-1121,woman's western shirt black xl 211-1102,"xhilaration Womens' Long Sleeve Sleep Shirt, Bubble Gum Pink, X-Large","xhilaration Womens' Long Sleeve Sleep Shirt, Purple/Raccoon, Medium",Éclair Women's Printed Thin Strap Blouse Black - Junior XL
review_userId,,,,,,,,,,,,,,,,,,,,,
A04728973KCBB7PLQB2BC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A05590483FAOIB9CFW4A9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A057523234N4Y49RPS4G2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A10076813OCUIPC1QBYZD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1007NP5HDQ8N7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AZWJR1508PTTD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZXWPBQC27BYY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AZY065T0PVYO7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
cosine_similarity(user_matrix).shape

(6670, 6670)

In [7]:
cosine_similarity(user_matrix.T).shape

(6584, 6584)

In [8]:
def similar_users (matrix,review_userId , k):
    user = matrix[matrix.index == review_userId]
    others = matrix[matrix.index != review_userId]
    #print(user)
    #print(others)
    similarity_scores = cosine_similarity(user, others)[0].tolist()
    others_index = others.index.tolist()
    user_similarity = list(zip(others_index, similarity_scores))
    sorted_scores = sorted(user_similarity, key = lambda x: x[1], reverse = True)
    similar_users = [u[0] for u in sorted_scores][:k]
    return similar_users

In [9]:
similar_users(user_matrix, 'A10076813OCUIPC1QBYZD', 5)

['A1040ISNVF9R69',
 'A10A2ZTH8JB9MI',
 'A10EGCJ6RZIUSU',
 'A10F32KLUDO3RU',
 'A10TEUPCDG7W81']

In [10]:
def recommend (review_userId, similar_users, matrix, items = 5):
    sim_usermatrix = matrix[matrix.index.isin(similar_users)]
    sim_users_avgratings = sim_usermatrix.mean(axis = 0)
    sim_usersdf = pd.DataFrame(sim_users_avgratings, columns = ['Mean'])
 
    user_df = matrix[matrix.index == review_userId]
    user_df_transposed = user_df.T
    user_df_transposed.columns = ['rating']
    user_unseen_movies = user_df_transposed[user_df_transposed['rating']==0]
    unseen_movies_indices = user_unseen_movies.index.tolist()
    
    similar_users_ratings = sim_usersdf[sim_usersdf.index.isin(unseen_movies_indices)]
    similar_users_ratings =  similar_users_ratings.sort_values(by =['Mean'], ascending = False)
    similar_users_ratings = similar_users_ratings.head(items)
    cloths_id = similar_users_ratings.index.tolist()
    print(cloths_id)

In [12]:
recommend('A10076813OCUIPC1QBYZD',['A1040ISNVF9R69', 'A10A2ZTH8JB9MI', 'A10EGCJ6RZIUSU', 'A10F32KLUDO3RU', 'A10TEUPCDG7W81'], user_matrix )

['"I Workout To Burn off The Crazy", Tri Blend Tank Top, Funny Workout Fitness Shirt Sleeveless for Women (2XL, Shocking Pink)', "Oddi Women's Short Sleeve Tye Dye Top Rayon Tank Top", 'Olivia Moon Navy Womens Small Cowl Neck Stretch Knit Top Blue S', 'Olivia Moon Mixed Media Tab Sleeve Top Blue Size Medium Petite', "Olivia Moon Gray Women's Small Striped Zipped Knit Top Blue S"]
